In [ ]:
# Draws ground truth yolo bboxes on a video

import numpy as np
import cv2
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import re

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
    return sorted(data, key=alphanum_key)

def vid2frame(path_in):
    old_wd = os.getcwd()
    vidcap = cv2.VideoCapture(path_in)
    # success, image = vidcap.read()
    image = vidcap.read()[1]
    frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    path_out = path_in.replace(".mp4", "_frames/")
    os.makedirs(path_out, exist_ok=True)
    for o in os.listdir(path_out):
        os.remove(os.path.join(path_out, o))
    os.chdir(path_out)
    print("Converting " + path_in + " to frames:")
    for m in tqdm(range(frames)):
        cv2.imwrite("%d.jpg" % m, image)
        # success, image = vidcap.read()
        image = vidcap.read()[1]
    vidcap.release()
    os.chdir(old_wd)


def frame2vid(path_in, path_out, fps):
    old_wd = os.getcwd()
    # path_out = path_in.strip("/") + ".mp4"
    frame_array = []
    files = [f for f in os.listdir(path_in) if os.path.isfile(os.path.join(path_in, f))]
    files = sorted_alphanumeric(files)
    print("Reading " + path_in + " for conversion to video:")
    for m in tqdm(range(len(files))):
        filename = path_in + files[m]
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        frame_array.append(img)
    out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*"MP4V"), fps, size)
    print("Converting " + path_in + " to video:")
    for m in tqdm(range(len(frame_array))):
        out.write(frame_array[m])
    out.release()
    os.chdir(old_wd)

In [ ]:
vid2frame("C:/Users/pv5/Desktop/test.mp4")

In [ ]:
ANNO_DIR = "C:/Users/pv5/Desktop/Elliot_gt/"
IMG_DIR = "C:/Users/pv5/Desktop/Elliot_frames/"
SAVE_DIR = "C:/Users/pv5/Desktop/Elliot_frames_gt/"
SAVE_PATH = "C:/Users/pv5/Desktop/Elliot_frames.mp4/"
os.makedirs(SAVE_DIR, exist_ok=True)
anno_names = sorted_alphanumeric(os.listdir(ANNO_DIR))
img_names = sorted_alphanumeric(os.listdir(IMG_DIR))

# assert len(anno_names) == len(img_names) #yolov5 outputs are missing some frames

print("Drawing GT bboxes:")
for i in tqdm(range(len(img_names))):
    annotations = np.loadtxt(ANNO_DIR + anno_names[i])
    img = cv2.imread(IMG_DIR + img_names[i])
    height = img.shape[0]
    width = img.shape[1]
    plt.imshow(img)
    if annotations.shape != (5,): #multiple objects
        for index in range(len(annotations)):
            obj_x_center, obj_y_center, obj_width, obj_height = annotations[index][1:]
            obj_x_center = int(width * obj_x_center)
            obj_y_center = int(height*obj_y_center)
            obj_width = int(width*obj_width)
            obj_height = int(height*obj_height)
            top_left_x = int(obj_x_center - obj_width / 2)
            top_left_y = int(obj_y_center + obj_height / 2)
            bottom_right_x = int(obj_x_center + obj_width / 2)
            bottom_right_y = int(obj_y_center - obj_height / 2)
            start_point = (top_left_x, top_left_y)
            end_point = (bottom_right_x, bottom_right_y)
            color = (0, 255, 0)
            thickness = 2
            img = cv2.rectangle(img, start_point, end_point, color, thickness)
    else: #single object
        obj_x_center, obj_y_center, obj_width, obj_height = annotations[1:]
        obj_x_center = int(width * obj_x_center)
        obj_y_center = int(height*obj_y_center)
        obj_width = int(width*obj_width)
        obj_height = int(height*obj_height)
        top_left_x = int(obj_x_center - obj_width / 2)
        top_left_y = int(obj_y_center + obj_height / 2)
        bottom_right_x = int(obj_x_center + obj_width / 2)
        bottom_right_y = int(obj_y_center - obj_height / 2)
        start_point = (top_left_x, top_left_y)
        end_point = (bottom_right_x, bottom_right_y)
        color = (0, 255, 0)
        thickness = 2
        img = cv2.rectangle(img, start_point, end_point, color, thickness)
            
    cv2.imwrite(SAVE_DIR+"%d.jpg" % i, img)

In [ ]:
SAVE_PATH = "C:/Users/pv5/Desktop/Elliot_test.mp4"
frame2vid(SAVE_DIR, SAVE_PATH, 24)